In [49]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [108]:
# importing our data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [109]:
train.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


<li> PassengerId - Numeric data, it is unique for each passenger
<li> Survived - Target data to be predicted, with incators 0 (Passenger Died) or 1 (Passenger Survived).
<li> Pclass - Numeric data, has 3 categories (1st, 2nd or 3rd)
<li> Name - String data type, mostly unique for each passenger
<li> Sex - Categorical data, has 2 categories (male, female)
<li> Age - Numeric data, max age is 80, it contains few nulls (can be imputed), useful.
<li> SibSp - Numeric data, max number of siblings / spouses aboard is 8, useful.
<li> Parch - Numeric data, max number of parents / children aboard is 6, useful.
<li> Ticket - Categorical data, it has a prefix to the number, it can be useful.
<li> Fare - Numeric data, higher the class, higher is the fare (need to check), 
        max fare is 512,32 (currency is $ I guess) , useful.
<li> Cabin - Categorical data, it contains few nulls (can be imputed), it can be useful.
<li> Embarked - Categorical data, with 3 categories, C = Cherbourg, Q = Queenstown, S = Southampton, 
        it contains few nulls (can be imputed), useful.

<h3>  Coisas para tomar cuidado na análise:</h3>
<li>1. valores faltantes
<li>2. valores muito grandes ou muito pequenos - outliers
<li>3. linhas duplicadas

Missining Values

In [110]:
#how many nulls are there?
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [111]:
#getting all nan for ages
train[train['Age'].isnull()].sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
354,355,0,3,"Yousif, Mr. Wazli",male,NaN,0,0,2647,7.225,NaN,C
95,96,0,3,"Shorney, Mr. Charles Joseph",male,NaN,0,0,374910,8.050,NaN,S


In [112]:
'''
algumas estratégias:
a. usar a média
b. usar a mediana
c. um modelo para predizer a idade
d. utlizar o nome Mrs ou Mr. = casada = Não é criança
e. ~ 
'''

train.Age.fillna(train.Age.mean(), inplace = True )
test.Age.fillna(train.Age.mean(), inplace = True )

In [113]:
# train['old'] = train.Name.apply(lambda x: 1 if any(j in x for j in ['Mr','Mrs']) else 0 )
# print(train[['old', 'Name']].sample(5))
# train.drop(columns=['old'], inplace= True)

Outliers

In [114]:
# três desvio padrão, para um curva normal, pode ser outlier.
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Duplicates

In [115]:
#do we have any duplicates?
train.Name.nunique() == len(train.Name)
#nops =D

True

In [116]:
train.PassengerId.nunique() == len(train.PassengerId)

True

<h1> Feature Engeneering

In [117]:
train['children'] = train.Age.apply(lambda x: 1 if x < 15 else 0)
test['children'] = test.Age.apply(lambda x: 1 if x < 15 else 0)

In [118]:
train.sample(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,children
227,228,0,3,"Lovell, Mr. John Hall (""Henry"")",male,20.5,0,0,A/5 21173,7.2500,NaN,S,0
271,272,1,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0000,NaN,S,0
287,288,0,3,"Naidenoff, Mr. Penko",male,22.0,0,0,349206,7.8958,NaN,S,0
233,234,1,3,"Asplund, Miss. Lillian Gertrud",female,5.0,4,2,347077,31.3875,NaN,S,1


In [119]:
# outras opções: clustering -> knn, gmm, etc

Dummies

In [120]:
train = pd.get_dummies(train, columns=['Sex', 'Pclass'])
test = pd.get_dummies(test, columns=['Sex', 'Pclass'])

In [121]:
train.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'children', 'Sex_female', 'Sex_male',
       'Pclass_1', 'Pclass_2', 'Pclass_3'],
      dtype='object')

In [122]:
#finalmente

In [123]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [124]:
y_columns = ['Survived']

x_columns = ['Age', 'SibSp', 'Parch', 'Fare', 'children',
             'Sex_female', 'Sex_male',
             'Pclass_1', 'Pclass_2', 'Pclass_3']

x_train,x_val, y_train, y_val = train_test_split(train[x_columns], 
                                                 train[y_columns],
                                                 test_size=0.2,
                                                 random_state=42)

In [159]:
model = RandomForestClassifier(n_estimators=100,
                                 max_depth=5, 
                                 random_state=42,
                                 oob_score = True,
                                criterion= 'entropy'
                           
                              )
clf = model.fit(x_train,y_train)

/home/willy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [160]:
clf.score(x_val, y_val)

0.8100558659217877

In [200]:
cross_val_score(estimator=clf, X=(x_val), y= y_val.values.ravel(), cv=10, verbose = 0).mean()

0.7994840041279669

In [214]:
print(pd.crosstab(np.array(y_val).reshape(1,-1)[0], clf.predict(x_val) , rownames=['Actual'], colnames=['Predicted'],margins=True))

Predicted    0   1  All
Actual                 
0           94  11  105
1           23  51   74
All        117  62  179


In [218]:
from sklearn.feature_selection import RFECV


selector = RFECV(clf, step=1, cv=5, n_jobs=-1)

selector = selector.fit(x_train, y_train)


print(pd.DataFrame(selector.ranking_, index = x_columns, columns=['importance']).sort_values('importance', ascending=True))


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


            importance
Age                  1
SibSp                1
Parch                1
Fare                 1
children             1
Sex_female           1
Sex_male             1
Pclass_1             1
Pclass_2             1
Pclass_3             1
